In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from datetime import datetime, timedelta
from order.models import *
from django.db.models import Avg, Sum, Count, Q, F

# 1. Models + CRUD operations

*ใน PART 1 นี้ใช้ไฟล์ข้อมูล order-1.sql*

**ส่งใน onlearn.it.kmitl.ac.th และเปลี่ยนชื่อไฟล์เป็นรหัสนักศึกษาเช่น 6700001.ipynb**

1.1 เพิ่ม model `Payment` สำหรับบันทึกการชำระเงิน โดยมี fields ดังต่อไปนี้ (1 คะแนน)
- order - one-to-one relationship with `Order`
- tip - เป็น Decimal field ซึ่งเก็บจำนวนทิป เป็นเลข 10 หลัก ทศนิยม 2 ตำแหน่ง และมีค่าตั้งต้นเป็น 0
- amount - เป็น Decimal field ซึ่งเก็บจำนวนยอดเงินรับชำระ เป็นเลข 10 หลัก ทศนิยม 2 ตำแหน่ง และมีค่าตั้งต้นเป็น 0
- payment_date - เป็น DateTime บันทึกวัน-เวลาที่ทำการสร้าง payment บันทึกเวลาปัจจุบันอัตโนมัติเมื่อมีการสร้าง Payment
- payment_method - เป็น ENUM field บันทึกช่องทางการชำระมี 3 ค่า CASH, QR, CREDIT และมีค่าตั้งต้นเป็น CASH

In [ ]:
# Copy code ของ model Payment มาแสดงตรงนี้ (ไม่ต้อง run cell นี้)
class Payment(models.Model):
    order = models.OneToOneField(Order, on_delete=models.PROTECT)
    tip = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    amount = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    payment_date = models.DateTimeField(auto_now_add=True)

    class PaymentMethod(models.TextChoices):
        CASH = 'CASH',
        QR = 'QR',
        CREDIT = 'CREDIT'

    payment_method = models.CharField(
        max_length=6,
        choices=PaymentMethod,
        default=PaymentMethod.CASH
    )

1.2 เพิ่ม model `MainIngredient` สำหรับบันทึกส่วนประกอบของเมนูแต่ละเมนู โดยมี fields ดังต่อไปนี้ (1 คะแนน)
- name - เก็บข้อมูลชื่อส่วนประกอบไม่เกิน 100 ตัวอักษร ห้ามเป็น NULL
- description - เก็บข้อมูลอธิบายเพิ่มเติมไม่เกิน 200 ตัวอักษร ค่าเป็น NULL ได้

**model MainIngredient มีความสัมพันธ์แบบ many-to-many กับ MenuItem**

In [ ]:
# Copy code ของ model MainIngredient มาแสดงตรงนี้ (ไม่ต้อง run cell นี้)
class MainIngredient(models.Model):
    name = models.CharField(max_length=100, null=False)
    description = models.CharField(max_length=200, null=True)
    menu_item = models.ManyToManyField(MenuItem)

    def __str__(self):
        return self.name

1.3 ให้เพิ่มลูกค้า (`Customer`) โดยมีข้อมูลดังนี้ (1 คะแนน)

- first_name: Stephen
- last_name: Curry
- email: s.curry30@gmail.com
- phone_number: 089-777-6655

In [2]:
# CODE HERE
c13 = Customer.objects.create(
    first_name = 'Stephen',
    last_name = 'Curry',
    email='s.curry30@gmail.com',
    phone_number = '089-777-6655'
)


In [3]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
cust = Customer.objects.get(first_name="Stephen")
print(cust)

Stephen Curry


1.4 เพิ่ม method `display_phone_number()` ใน model `Customer` สำหรับแสดงข้อมูลเบอร์โทรศัพท์ โดยตัดเครื่องหมาย '-' ออก และเติม '+66' ไปด้านหน้า ซึ่งจะต้องแสดงผลดังนี้: '+66897776655' (1 คะแนน)

*Hint: เมื่อเขียน method ใน model `Customer` เสร็จแล้ว จะต้อง restart Jupyter Notebook หนึ่งครั้ง*

In [ ]:
# Copy code ของ method display_phone_number() มาแสดงตรงนี้ (ไม่ต้อง run cell นี้)
def display_phone_number(self):
    first = self.phone_number[1:3]
    center = self.phone_number[4:7]
    last = self.phone_number[8:]

    return f"+66{first}{center}{last}"

In [8]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
for customer in Customer.objects.all():
    print(customer.display_phone_number())

+66834567890
+66825678901
+66896789012
+66817890123
+66897776655


1.5 เพิ่มข้อมูล `MainIngredient` สำหรับ `MenuItem` โดยเพิ่มข้อมูลดังนี้ (1 คะแนน)
- MenuItem: 'Lemonade'
    - ส่วนประกอบที่ 1 => name: Lemon, description: Organic fresh lemon from our farm
    - ส่วนประกอบที่ 2 => name: Sugar, description: None
- MenuItem: 'Caesar Salad'
    - ส่วนประกอบที่ 1 => name: Romaine lettuce, description: None
    - ส่วนประกอบที่ 2 => name: Croutons, description: None
    - ส่วนประกอบที่ 3 => name: Lemon, description: Organic fresh lemon from our farm **(ต้องเป็นตัวเดิมที่สร้างตอนทำของ Lemonade)**
    - ส่วนประกอบที่ 4 => name: Parmesan cheese, description: Cheese allergy

In [5]:
# CODE HERE
from order.models import MainIngredient


mi151 = MenuItem.objects.get(name='Lemonade')

lemonade1 = MainIngredient.objects.create(
    name='Lemon', 
    description='Organic fresh lemon from our farm'
)

lemonade2 = MainIngredient.objects.create(
    name='Sugar'
)

lemonade1.menu_item.add(mi151)
lemonade2.menu_item.add(mi151)


##################
mi152 = MenuItem.objects.get(name='Caesar Salad')

salad1 = MainIngredient.objects.create(
    name='Romaine lettuce'
)

salad2 = MainIngredient.objects.create(
    name="Croutons"
)

salad3 = MainIngredient.objects.get(name='Lemon') 
salad4 = MainIngredient.objects.create(
    name="Parmesan cheese", 
    description='Cheese allergy'
) 

salad1.menu_item.add(mi152)
salad2.menu_item.add(mi152)
salad3.menu_item.add(mi152)
salad4.menu_item.add(mi152)


In [13]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
for menu in MenuItem.objects.filter(name__in=["Lemonade", "Caesar Salad"]):
    print(f"MENU: {menu.name}")
    for ingredient in menu.mainingredient_set.all():
        print(f"- {ingredient.name} ({ingredient.description or 'N/A'})")

MENU: Caesar Salad
- Lemon (Organic fresh lemon from our farm)
- Romaine lettuce (N/A)
- Croutons (N/A)
- Parmesan cheese (Cheese allergy)
MENU: Lemonade
- Lemon (Organic fresh lemon from our farm)
- Sugar (N/A)


1.6 ให้สร้าง `Order` ที่มีรายการ 'Margherita Pizza', 'Caesar Salad', 'Chocolate Lava Cake' และ 'Coffee' อย่างละ 1 ที่สั่งโดยคุณ Stephen Curry (1 คะแนน)

In [6]:
# CODE HERE
stephen = Customer.objects.get(first_name='Stephen', last_name='Curry')

order = Order.objects.create(
    customer=stephen
)

pizzamenu = MenuItem.objects.get(name='Margherita Pizza')
saladmenu = MenuItem.objects.get(name='Caesar Salad')
cakemenu = MenuItem.objects.get(name='Chocolate Lava Cake')
coffeemenu = MenuItem.objects.get(name='Coffee')

orderitem1 = OrderItem.objects.create(
    order=order,
    menu_item=pizzamenu
)

orderitem2 = OrderItem.objects.create(
    order=order,
    menu_item=saladmenu
)

orderitem3 = OrderItem.objects.create(
    order=order,
    menu_item=cakemenu
)

orderitem4 = OrderItem.objects.create(
    order=order,
    menu_item=coffeemenu
)


In [16]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
order = Order.objects.get(customer__first_name="Stephen")
print(f"CUST: {order.customer}, ORDER DATE: {order.order_date.strftime(format='%Y-%m-%d %H:%M')}")
for order_item in order.orderitem_set.all():
    print(f"- {order_item.menu_item.name}: {order_item.quantity}")

CUST: Stephen Curry, ORDER DATE: 2025-08-28 07:54
- Margherita Pizza: 1
- Caesar Salad: 1
- Chocolate Lava Cake: 1
- Coffee: 1


1.7 ต่อเนื่องจากข้อ 1.6 คุณ Stephen Curry มีการขอแก้ไข `Order` ที่สั่งไปจาก 'Coffee' 1 ที่ เป็น 'Lemonade' 2 ที่ และยกเลิกรายการ 'Chocolate Lava Cake' (1 คะแนน)

In [ ]:
# CODE HERE
editorder = OrderItem.objects.get(
    order__customer__first_name="Stephen", 
    order__customer__last_name='Curry', 
    menu_item__name='Coffee'
)

editorder.quantity = 1
lemonade = MenuItem.objects.get(name='Lemonade')
editorder.menu_item = lemonade

editorder.menu_item.name
editorder.quantity

editorder.save()

editorder2 = OrderItem.objects.get(
    order__customer__first_name="Stephen", 
    order__customer__last_name='Curry', 
    menu_item__name='Chocolate Lava Cake'
).delete()

In [28]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
order = Order.objects.get(customer__first_name="Stephen")
print(f"CUST: {order.customer}, ORDER DATE: {order.order_date.strftime(format='%Y-%m-%d %H:%M')}")
for order_item in order.orderitem_set.all():
    print(f"- {order_item.menu_item.name}: {order_item.quantity}")

CUST: Stephen Curry, ORDER DATE: 2025-08-28 07:54
- Margherita Pizza: 1
- Caesar Salad: 1
- Lemonade: 1


1.8 จากนั้นคุณ Stephen Curry ทำการเรียกชำระเงินออเดอร์ในข้อ 1.7 และมีการให้ทิป 2.00 USD จงทำการสร้างข้อมูลใน `Payment` (`payment_method` เป็น QR) (2 คะแนน)

*หมายเหตุ 1: Payment.amount จะเป็นยอดชำระสุทธิ ค่าอาหาร เครื่องดื่มทั้งหมด + ค่าทิป*

*หมายเหตุ 2: การคำนวณยอดรวมให้คำนวณโดยใช้ aggregation `Sum` ที่คำนวณจาก `OrderItem` ... ห้ามบวกลบยอดเงินเองข้างนอก*

In [41]:
# CODE HERE
stephen = Order.objects.get(customer__first_name='Stephen', customer__last_name='Curry')

totalOrderItem1 = OrderItem.objects.filter(order=stephen)

# totalOrderItem1

total = 0
for i in totalOrderItem1:
    total += i.menu_item.price
    
# print(total)
    
payment = Payment.objects.create(
    order=stephen,
    tip=2,
    amount=total+2
)



20.47


In [42]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
order = Order.objects.get(customer__first_name="Stephen")
payment = order.payment
print(f"PAYMENT DATE: {payment.payment_date}, TIP: {payment.tip}, AMOUNT: {payment.amount}, METHOD: {payment.payment_method}")

PAYMENT DATE: 2025-08-28 08:24:19.225586+00:00, TIP: 2.00, AMOUNT: 22.47, METHOD: CASH


1.9 ให้ค้นหาออเดอร์ (`Order`) ของเดือน 9 ปี 2025 และให้ปรับสถานะเป็น False ทั้งหมด (1 คะแนน)

In [48]:
# CODE HERE - Filter ออเดอร์ของเดือน 9 ปี 2025
ordermonth9 = Order.objects.filter(order_date__month=9, order_date__year=2025)
ordermonth9.values()


<QuerySet [{'id': 9, 'customer_id': 1, 'order_date': datetime.datetime(2025, 9, 1, 5, 15, tzinfo=datetime.timezone.utc), 'is_completed': False}, {'id': 10, 'customer_id': 2, 'order_date': datetime.datetime(2025, 9, 3, 13, 0, tzinfo=datetime.timezone.utc), 'is_completed': True}, {'id': 17, 'customer_id': 2, 'order_date': datetime.datetime(2025, 9, 10, 4, 15, tzinfo=datetime.timezone.utc), 'is_completed': False}, {'id': 18, 'customer_id': 4, 'order_date': datetime.datetime(2025, 9, 15, 12, 0, tzinfo=datetime.timezone.utc), 'is_completed': True}, {'id': 19, 'customer_id': 1, 'order_date': datetime.datetime(2025, 9, 20, 9, 30, tzinfo=datetime.timezone.utc), 'is_completed': False}]>

In [54]:
# CODE HERE - Update `is_completed` = False
ordermonth9 = ordermonth9.filter(is_completed=True)
ordermonth9.values()

for i in ordermonth9:
    i.is_completed=False
    i.save()

ordermonth9.values()

sep_orders = Order.objects.filter(order_date__month=9, order_date__year=2025)


In [55]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
# ให้ตั้งชื่อตัวแปรที่เก็บข้อมูลออเดอร์ของเดือน 9 ปี 2025 ชื่อว่า sep_orders
for order in sep_orders:
    print(f"CUST: {order.customer}, DATE: {order.order_date.strftime(format='%Y-%m-%d')}, IS COMPLETED?: {bool(order.is_completed)}")

CUST: John Doe, DATE: 2025-09-01, IS COMPLETED?: False
CUST: Jane Smith, DATE: 2025-09-10, IS COMPLETED?: False
CUST: John Doe, DATE: 2025-09-20, IS COMPLETED?: False
CUST: Jane Smith, DATE: 2025-09-03, IS COMPLETED?: False
CUST: Bob Williams, DATE: 2025-09-15, IS COMPLETED?: False


# 2. Making Queries

*สำหรับ PART 2 - ให้ทำการ reset DB และ import ข้อมูลใน order-2.sql เข้าไปใหม่*

2.1 แสดงรายการออเดอร์ (`Order`) และจำนวนจานของอาหาร/เครื่องดื่มที่สั่ง (`OrderItem.quantity`) ทั้งหมดของคุณ 'John Doe' เรียนจากจำนวนจากจากมาก-น้อย (1 คะแนน)

**Expected Output**

```text
ORDER ID: 17, CUSTOMER: John Doe, QUANTITY: 9
ORDER ID: 24, CUSTOMER: John Doe, QUANTITY: 8
ORDER ID: 30, CUSTOMER: John Doe, QUANTITY: 4
ORDER ID: 29, CUSTOMER: John Doe, QUANTITY: 3
ORDER ID: 1, CUSTOMER: John Doe, QUANTITY: 3
ORDER ID: 11, CUSTOMER: John Doe, QUANTITY: 2
```

In [23]:
# CODE HERE
from django.db.models import Avg, Sum, Count, Q, F

order21 = Order.objects.filter(
    customer__first_name='John', customer__last_name='Doe'
).distinct().annotate(quantity=Sum('orderitem__quantity')).order_by('-quantity')

order21
for i in order21:
    print(f'ORDER ID: {i.id}, CUSTOMER: {i.customer}, QUANTITY: {i.quantity}')

ORDER ID: 17, CUSTOMER: John Doe, QUANTITY: 9
ORDER ID: 24, CUSTOMER: John Doe, QUANTITY: 8
ORDER ID: 30, CUSTOMER: John Doe, QUANTITY: 4
ORDER ID: 1, CUSTOMER: John Doe, QUANTITY: 3
ORDER ID: 29, CUSTOMER: John Doe, QUANTITY: 3
ORDER ID: 11, CUSTOMER: John Doe, QUANTITY: 2


2.2 ให้หาจำนวนออเดอร์ (`Order`) ที่มีรายการอาหาร (`OrderItem`) มากกว่า 1 รายการขึ้นไป และสั่ง 'Cheeseburger' (1 คะแนน)

**Expected Output**

รายการออเดอร์มีมากกว่า 1 รายการและสั่ง Cheeseburger มี 4 ออเดอร์

In [25]:
# CODE HERE
############################### เดี๋ยวกลับมานะ
# order22 = Order.objects.filter(orderitem__menu_item__name= 'Cheeseburger')

# order22.values()

<QuerySet [{'id': 2, 'customer_id': 4, 'order_date': datetime.datetime(2025, 6, 1, 2, 13, tzinfo=datetime.timezone.utc), 'is_completed': True}, {'id': 13, 'customer_id': 3, 'order_date': datetime.datetime(2025, 7, 1, 2, 59, tzinfo=datetime.timezone.utc), 'is_completed': False}, {'id': 21, 'customer_id': 2, 'order_date': datetime.datetime(2025, 6, 14, 8, 30, tzinfo=datetime.timezone.utc), 'is_completed': True}, {'id': 24, 'customer_id': 1, 'order_date': datetime.datetime(2025, 8, 28, 4, 25, tzinfo=datetime.timezone.utc), 'is_completed': True}]>

2.3 ให้แสดงรายชือลูกค้า (`Customer`) โดยเรียงจาก ลูกค้าที่ยอดใช้จ่ายกับทางร้านมาก - น้อย (1 คะแนน)

*Hint: ยอดใช้จ่ายให้คำนวณจาก `Payment.amount`*

**Expected Output**

```text
CUSTOMER: Alice Johnson, TOTAL SPENDING: 325.47
CUSTOMER: John Doe, TOTAL SPENDING: 258.75
CUSTOMER: Jane Smith, TOTAL SPENDING: 258.08
CUSTOMER: Bob Brown, TOTAL SPENDING: 171.68
CUSTOMER: Carol Davis, TOTAL SPENDING: 137.33
```

In [37]:
# CODE HERE
topspend = Customer.objects.all().annotate(top=Sum('order__payment__amount')).order_by('-top')

for o in topspend:
    print(f'CUSTOMER: {o}, TOTAL SPENDING: {o.top}')


CUSTOMER: Alice Johnson, TOTAL SPENDING: 325.47
CUSTOMER: John Doe, TOTAL SPENDING: 258.75
CUSTOMER: Jane Smith, TOTAL SPENDING: 258.08
CUSTOMER: Bob Brown, TOTAL SPENDING: 171.68
CUSTOMER: Carol Davis, TOTAL SPENDING: 137.33


2.4 ให้แสดงเมนูอาหาร (`MenuItem`) ทั้งหมดของร้าน พร้อมทั้ง (`MainIngredient`) (1 คะแนน)

**Expected Output**

```text
MENU: Cheeseburger, INGREDIENTS: Beef, Cheddar Cheese
MENU: Vegan Salad, INGREDIENTS: Tomato, Lettuce
MENU: Spaghetti Bolognese, INGREDIENTS: Beef, Mozzarella
MENU: Grilled Chicken, INGREDIENTS: Chicken Breast
MENU: Pizza Margherita, INGREDIENTS: Tomato, Mozzarella
MENU: Caesar Salad, INGREDIENTS: Lettuce
MENU: Fish & Chips, INGREDIENTS: Beef
MENU: Pancakes, INGREDIENTS: Lettuce
MENU: Taco Plate, INGREDIENTS: Beef, Rice
MENU: Ice Cream Sundae, INGREDIENTS: Cheddar Cheese, Flour
```

In [51]:
# CODE HERE
allmenuitem = MenuItem.objects.all().annotate(mi=F('mainingredient__name'))
# allmenuitem = MenuItem.objects.all()

allmenuitem
for i in allmenuitem:
    print(f'MENU: {i.name}, INGREDIENTS: {i.mi}')


MENU: Cheeseburger, INGREDIENTS: Beef
MENU: Cheeseburger, INGREDIENTS: Cheddar Cheese
MENU: Vegan Salad, INGREDIENTS: Tomato
MENU: Vegan Salad, INGREDIENTS: Lettuce
MENU: Spaghetti Bolognese, INGREDIENTS: Beef
MENU: Spaghetti Bolognese, INGREDIENTS: Mozzarella
MENU: Grilled Chicken, INGREDIENTS: Chicken Breast
MENU: Pizza Margherita, INGREDIENTS: Tomato
MENU: Pizza Margherita, INGREDIENTS: Mozzarella
MENU: Caesar Salad, INGREDIENTS: Lettuce
MENU: Fish & Chips, INGREDIENTS: Beef
MENU: Pancakes, INGREDIENTS: Lettuce
MENU: Taco Plate, INGREDIENTS: Beef
MENU: Taco Plate, INGREDIENTS: Rice
MENU: Ice Cream Sundae, INGREDIENTS: Cheddar Cheese
MENU: Ice Cream Sundae, INGREDIENTS: Flour


2.5 แสดงเมนูอาหาร (`MenuItem`) ที่มีราคามากกว่า 10 USD หรือ มีคำว่า "Fresh" ในรายละเอียดของส่วนประกอบ (`MainIngredient.description`) (1 คะแนน)

**Expected Output**

```text
MENU: Pizza Margherita, DESCRIPTION: Tomato and mozzarella pizza.
MENU: Vegan Salad, DESCRIPTION: Fresh mixed greens with seasonal vegetables.
MENU: Fish & Chips, DESCRIPTION: Crispy fried fish with fries.
```

In [65]:
# CODE HERE
menuitem25 = MenuItem.objects.filter(Q(price__gt=10) | Q(mainingredient__description__icontains='Fresh'))

# menuitem25
for i in menuitem25:
    print(f'MENU: {i}, DESCRIPTION: {i.description}')

MENU: Vegan Salad, DESCRIPTION: Fresh mixed greens with seasonal vegetables.
MENU: Pizza Margherita, DESCRIPTION: Tomato and mozzarella pizza.
MENU: Pizza Margherita, DESCRIPTION: Tomato and mozzarella pizza.
MENU: Fish & Chips, DESCRIPTION: Crispy fried fish with fries.


2.6 แสดงเมนูอาหารที่มียอดขาย (ดูจาก `OrderItem.quantity` * `MenuItem.price`) สูงสุด 3 ลำดับแรก เรียงจากมาก - น้อย (1 คะแนน)

**Expected Output**

```text
MENU: Grilled Chicken, SALES: 195.00
MENU: Vegan Salad, SALES: 149.50
MENU: Pizza Margherita, SALES: 143.00
```

In [79]:
# CODE HERE
# bestseller = OrderItem.objects.annotate(d=F('quantity')*F('menu_item__price')).order_by('-d')[:3]
# น่าจะต้องเอา orderitem ไป countไหม ไม่รู้ เดี๋ยวกลับมา

bestseller = MenuItem.objects.annotate(d=F('orderitem__quantity')*F('price')).order_by('-d')[:3]

for i in bestseller:
    print(f'{i} {i.d}')

Fish & Chips 39.00
Spaghetti Bolognese 30.00
Grilled Chicken 29.25


2.7 แสดงจำนวนทิป (`Payment.tip`) ที่ได้รับสำหรับการชำระเงินในแต่ละวิธี (`Payment.payment_method`) เรียงลำดับตามจำนวนทิปมาก - น้อย (1 คะแนน)

*Hint: ให้ทำ annotate/aggregate ค่าทิป โดย group by ด้วย `Payment.payment_method`*

**Expected Output**

```text
MONTH: CASH, TOTAL TIP: 31.05
MONTH: CREDIT, TOTAL TIP: 25.58
MONTH: QR, TOTAL TIP: 21.79
```

In [14]:
# CODE HERE
# tip = Payment.objects.annotate(tips=Sum('tip')).values(
#     'payment_method'
# )

# for i in tip:
#     print(f'{i}')


TypeError: QuerySet.annotate() received non-expression(s): payment_method.

2.8 แสดงรายชื่อลูกค้า (`Customer`) ทั้งหมด โดยแสดงข้อมูลวันที่เข้าใช้บริการล่าสุด (`Order.order_date`) (โดยนับเฉพาะออเดอร์ที่สมบูรณ์แล้ว - `Order.is_completed` = True) (2 คะแนน)

*Hint: จะต้องใช้ Subquery*

**Expected Output**

```text
CUSTOMER: John Doe, LASTEST VISIT: 2025-08-28 04:25
CUSTOMER: Jane Smith, LASTEST VISIT: 2025-08-03 01:42
CUSTOMER: Alice Johnson, LASTEST VISIT: 2025-08-26 11:33
CUSTOMER: Bob Brown, LASTEST VISIT: 2025-08-19 02:04
CUSTOMER: Carol Davis, LASTEST VISIT: 2025-07-26 01:48
```

In [ ]:
from django.db.models import OuterRef, Subquery
# CODE HERE


2.9 แสดงข้อมูลสำหรับการออกใบเสร็จรับเงินของ payment ที่ ID = 10 โดยจัดรูปแบบดังตัวอย่าง (1 คะแนน)

*Hint: สร้างเป็น dict เองได้เลยนะครับ ทำง่ายๆ ไม่ต้องยาก แค่เอาค่ามาแสดงให้ถูกต้อง*

**Example Output (ตัวอย่างนี้ไม่ใช่ผลลัพธ์ของ payment ID 10 นะครับ)**

```python
{
    "customer": "John Doe",
    "email": "john.doe@example.com",
    "phone_number": "083-456-7890",
    "payment_method": "QR Promtpay",
    "tip": 2.00,
    "amount": 27.48,
    "payment_date": "2025-01-10",
    "payment_time": "11:30",
    "duration": "60" # จำนวนนาที โดยคิดจาก payment.payment_date - order.order_date (เศษวินาทีปัดทิ้งเลย)
    "order_items": [
        {
            "name": "Cheeseburger",
            "description": None,
            "quantity": 2,
            "price": 8.99
        },
        {
            "name": "Caesar Salad",
            "description": None,
            "quantity": 1,
            "price": 7.5
        },
    ]
}
```

In [ ]:
# CODE HERE
